In [14]:
import keras_nlp
from transformers import BertTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from official.nlp import optimization
import tensorflow_text as text

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_ = "Preparing data for BERT"
tokens = tokenizer.tokenize(text_)

In [3]:
input_id = tokenizer.encode(text_, add_special_tokens=True)

In [4]:
attention_mask = [1] * len(input_id)

In [5]:
attention_mask

[1, 1, 1, 1, 1, 1]

In [6]:
data_folder = '../../data/corpus_for_sexism_french/'

In [7]:
df = pd.read_csv(f'{data_folder}corpus_with_text_label_final.csv').iloc[:, 1:].dropna()
df

,full_text,label
1,@MamzelleMNa Une très humble femme! #Ironie,0
2,"BLOGUE - «Tsé, la parité homme-femme...» au se...",0
5,Mise en ligne de mon article sur @PayeTaShnek ...,0
7,Hommage #journeedelafemme Toute sa vie elle a ...,0
8,Lustre bois flotté feuilles exotiques bleues a...,0
...,...,...
9545,"[Exclusif] #BalanceTonPorc, transparence, juge...",1
9547,"Eric Monier, directeur de la rédaction de #LCI...",1
9548,.@annesaurat porte plainte contre son ancien r...,1
9550,"Moïra Sauvage @excisionparlons ""une pensée pou...",1


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['full_text'], df['label'], stratify=df['label'])

In [9]:
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

2024-06-02 14:17:36.138256: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-02 14:17:36.377127: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [12]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
#model = tf.keras.Model(inputs=[text_input], outputs = [l])
#model.fit(X_train, y_train, epochs=10)


ValueError: Exception encountered when calling layer 'keras_layer_2' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_2' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=None, name=text>
  • training=None

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_small_en_uncased",trainable=True)
encoder_inputs = preprocessor(text_input)
encoder = keras_nlp.models.BertBackbone.from_preset("bert_small_en_uncased")
outputs = encoder(encoder_inputs)

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs = [l])

loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

model.compile(loss=loss)
model.fit(X_train, y_train, epochs=10)

Epoch 1/10


/home/arys/miniconda3/envs/DEV_IA/lib/python3.10/site-packages/keras/src/backend/tensorflow/nn.py:674: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


75/75 ━━━━━━━━━━━━━━━━━━━━ 962s 13s/step - loss: 0.7882
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 942s 13s/step - loss: 0.4107
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 926s 12s/step - loss: 0.4108
Epoch 4/10
72/75 ━━━━━━━━━━━━━━━━━━━━ 37s 12s/step - loss: 0.4206